Esto ejecuta mínimos cuadrados cuando solo hay una línea y covarianza cuando no 

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate
from scipy.ndimage import binary_opening

# Filtrar puntos dispersos en la máscara
def clean_mask(mask):
    cleaned_mask = binary_opening(mask, structure=np.ones((3, 3)))  # Filtro morfológico
    return cleaned_mask

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Filtro por porcentaje de energía
def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)  # Energía total de la imagen en la frecuencia
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]  # Ordenamos de mayor a menor
    cumulative_energy = np.cumsum(sorted_magnitude)  # Suma acumulativa de energía
    
    # Encontramos el umbral donde alcanzamos el porcentaje de energía deseado
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]

    # Creamos la máscara conservando solo los valores con suficiente energía
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask  # Aplicamos el filtro
    
    print(f"Umbral de energía aplicado: {threshold_value:.4f} (Conserva {energy_percent}% de la energía)")
    return filtered_transform, mask

# Calcular la dirección principal del espectro filtrado
def calculate_main_direction(mask, magnitude=None):
    """
    Calcula la dirección principal del espectro filtrado.
    
    Parámetros:
        mask (ndarray): Máscara binaria del espectro.
        magnitude (ndarray, opcional): Magnitud del espectro para ponderar los puntos.
    
    Retorna:
        float: Dirección principal en grados.
    """
    h, w = mask.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Filtrar puntos dispersos
    mask = clean_mask(mask)

    # Coordenadas de los puntos activados en la máscara
    y, x = np.nonzero(mask)
    if len(x) == 0 or len(y) == 0:
        print("La máscara no tiene puntos activados.")
        return 0  # O algún valor predeterminado

    y = y - cy
    x = x - cx

    # Calcular la relación de aspecto de los puntos activados
    range_x = np.max(x) - np.min(x)
    range_y = np.max(y) - np.min(y)
    aspect_ratio = max(range_x, 1) / max(range_y, 1)  # Evitar división por cero

    # Depuración: Imprimir los valores
    print(f"range_x: {range_x}, range_y: {range_y}, aspect_ratio: {aspect_ratio}")

    # Decidir el método basado en la relación de aspecto
    if aspect_ratio > 1.5 or aspect_ratio < 0.7:  # Una sola línea preferente
        print("Usando método: least_squares")
        # Ajustar una línea usando mínimos cuadrados
        A = np.vstack([x, np.ones(len(x))]).T
        m, _ = np.linalg.lstsq(A, y, rcond=None)[0]
        main_direction = np.arctan(m)  # Ángulo en radianes
    else:
        print("Usando método: covariance")
        # Ponderar por la magnitud si se proporciona
        if magnitude is not None:
            magnitude = np.abs(magnitude)  # Asegurarse de que no haya valores negativos
            weights = magnitude[mask]
            weights /= np.sum(weights)  # Normalizar los pesos
            cov_matrix = np.cov(x, y, aweights=weights)
        else:
            cov_matrix = np.cov(x, y)
        
        # Calcular valores y vectores propios
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
        main_direction = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])  # Ángulo en radianes

    return np.degrees(main_direction)  # Convertir a grados


# Filtro de dos líneas delgadas rotadas
def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Crear una máscara con dos líneas delgadas
    mask = np.zeros((h, w), dtype=np.uint8)

    # Línea horizontal
    mask[cy - 1:cy + 2, :] = 1  # Grosor de 3 píxeles

    # Línea vertical
    mask[:, cx - 1:cx + 2] = 1  # Grosor de 3 píxeles

    # Rotar la máscara
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

# Función principal para procesar todas las imágenes en una carpeta
def process_images_with_rotated_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Transformada de Fourier 2D
        
        # Aplicar el filtro por energía al 20%
        filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, energy_percent=20)

        # Calcular la dirección principal del espectro filtrado
        main_direction = calculate_main_direction(mask_energy, magnitude=f_transform_shifted)
        print(f"Dirección principal calculada: {main_direction:.2f} grados")

        # Aplicar el filtro de dos líneas rotadas
        filtered_transform_lines, rotated_mask = filter_two_lines_rotated(f_transform_shifted, main_direction)

        # Espectros filtrados para visualizar
        filtered_magnitude_spectrum_energy = np.log1p(np.abs(filtered_transform_energy))
        filtered_magnitude_spectrum_lines = np.log1p(np.abs(filtered_transform_lines))
        
        # Reconstrucción de la imagen desde la transformada filtrada
        reconstructed_img_energy = np.abs(ifftn(filtered_transform_energy))
        reconstructed_img_lines = np.abs(ifftn(filtered_transform_lines))

        # Visualización de resultados
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(2, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(2, 4, 3)
        plt.imshow(mask_energy, cmap='gray')
        plt.title('Máscara del Filtro (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 4)
        plt.imshow(filtered_magnitude_spectrum_energy, cmap='inferno')
        plt.title('Espectro Filtrado (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 5)
        plt.imshow(rotated_mask, cmap='gray')
        plt.title(f'Máscara Rotada ({main_direction:.2f}°)')
        plt.axis('off')
        
        plt.subplot(2, 4, 6)
        plt.imshow(filtered_magnitude_spectrum_lines, cmap='inferno')
        plt.title('Espectro Filtrado (Líneas Rotadas)')
        plt.axis('off')
        
        plt.subplot(2, 4, 7)
        plt.imshow(reconstructed_img_energy, cmap='gray')
        plt.title('Imagen Reconstruida (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 8)
        plt.imshow(reconstructed_img_lines, cmap='gray')
        plt.title('Imagen Reconstruida (Líneas Rotadas)')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Ejecutar el procesamiento con filtro rotado
process_images_with_rotated_filter("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")

Ahora lo que he hecho es uno que siempre calcula las 3 direcciones principales con covarience para saber como se adaptan a cada imagen


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate
from scipy.ndimage import binary_opening

# Filtrar puntos dispersos en la máscara
def clean_mask(mask):
    cleaned_mask = binary_opening(mask, structure=np.ones((3, 3)))  # Filtro morfológico
    return cleaned_mask

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Filtro por porcentaje de energía
def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)  # Energía total de la imagen en la frecuencia
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]  # Ordenamos de mayor a menor
    cumulative_energy = np.cumsum(sorted_magnitude)  # Suma acumulativa de energía
    
    # Encontramos el umbral donde alcanzamos el porcentaje de energía deseado
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]

    # Creamos la máscara conservando solo los valores con suficiente energía
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask  # Aplicamos el filtro
    
    print(f"Umbral de energía aplicado: {threshold_value:.4f} (Conserva {energy_percent}% de la energía)")
    return filtered_transform, mask

# Calcular la dirección principal del espectro filtrado
def calculate_main_directions(mask, magnitude=None, num_directions=3):
    """
    Calcula las direcciones principales del espectro filtrado.
    
    Parámetros:
        mask (ndarray): Máscara binaria del espectro.
        magnitude (ndarray, opcional): Magnitud del espectro para ponderar los puntos.
        num_directions (int): Número de direcciones principales a calcular.
    
    Retorna:
        list: Lista de direcciones principales en grados.
    """
    h, w = mask.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Filtrar puntos dispersos
    mask = clean_mask(mask)

    # Coordenadas de los puntos activados en la máscara
    y, x = np.nonzero(mask)
    if len(x) == 0 or len(y) == 0:
        print("La máscara no tiene puntos activados.")
        return [0]  # O algún valor predeterminado

    y = y - cy
    x = x - cx

    # Calcular la matriz de covarianza
    if magnitude is not None:
        magnitude = np.abs(magnitude)  # Asegurarse de que no haya valores negativos
        weights = magnitude[mask]
        weights /= np.sum(weights)  # Normalizar los pesos
        cov_matrix = np.cov(x, y, aweights=weights)
    else:
        cov_matrix = np.cov(x, y)

    # Calcular valores y vectores propios
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    # Ordenar los valores propios y vectores propios en orden descendente
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Calcular las direcciones principales
    directions = []
    for i in range(min(num_directions, len(eigenvalues))):
        angle = np.arctan2(eigenvectors[1, i], eigenvectors[0, i])  # Ángulo en radianes
        directions.append(np.degrees(angle))  # Convertir a grados

    # Si no hay suficientes direcciones, rellena con direcciones uniformemente distribuidas
    while len(directions) < num_directions:
        directions.append((360 / num_directions) * len(directions))

    return directions

# Filtro de dos líneas delgadas rotadas
def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Crear una máscara con dos líneas delgadas
    mask = np.zeros((h, w), dtype=np.uint8)

    # Línea horizontal
    mask[cy - 1:cy + 2, :] = 1  # Grosor de 3 píxeles

    # Línea vertical
    mask[:, cx - 1:cx + 2] = 1  # Grosor de 3 píxeles

    # Rotar la máscara
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

# Función principal para procesar todas las imágenes en una carpeta
def process_images_with_rotated_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Transformada de Fourier 2D
        
        # Aplicar el filtro por energía al 20%
        filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, energy_percent=20)

        # Calcular las direcciones principales del espectro filtrado
        main_directions = calculate_main_directions(mask_energy, magnitude=np.abs(f_transform_shifted), num_directions=3)
        print(f"Direcciones principales calculadas: {main_directions}")

        # Aplicar el filtro de dos líneas rotadas para cada dirección
        combined_filtered_transform = np.zeros_like(f_transform_shifted)
        combined_rotated_mask = np.zeros_like(mask_energy, dtype=np.uint8)  # Asegurar que sea uint8
        for direction in main_directions:
            filtered_transform_lines, rotated_mask = filter_two_lines_rotated(f_transform_shifted, direction)
            combined_filtered_transform += filtered_transform_lines
            combined_rotated_mask += rotated_mask

        # Espectros filtrados para visualizar
        filtered_magnitude_spectrum_energy = np.log1p(np.abs(filtered_transform_energy))
        filtered_magnitude_spectrum_lines = np.log1p(np.abs(combined_filtered_transform))
        
        # Reconstrucción de la imagen desde la transformada filtrada
        reconstructed_img_energy = np.abs(ifftn(filtered_transform_energy))
        reconstructed_img_lines = np.abs(ifftn(combined_filtered_transform))

        # Visualización de resultados
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(2, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(2, 4, 3)
        plt.imshow(mask_energy, cmap='gray')
        plt.title('Máscara del Filtro (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 4)
        plt.imshow(filtered_magnitude_spectrum_energy, cmap='inferno')
        plt.title('Espectro Filtrado (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 5)
        plt.imshow(combined_rotated_mask, cmap='gray')
        plt.title(f'Máscara Rotada ({len(main_directions)} direcciones)')
        plt.axis('off')
        
        plt.subplot(2, 4, 6)
        plt.imshow(filtered_magnitude_spectrum_lines, cmap='inferno')
        plt.title('Espectro Filtrado (Líneas Rotadas)')
        plt.axis('off')
        
        plt.subplot(2, 4, 7)
        plt.imshow(reconstructed_img_energy, cmap='gray')
        plt.title('Imagen Reconstruida (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 8)
        plt.imshow(reconstructed_img_lines, cmap='gray')
        plt.title('Imagen Reconstruida (Líneas Rotadas)')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
# Ejecutar el procesamiento con filtro rotado
process_images_with_rotated_filter("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")

Esta combina ambas según la imagen (este es el bueno ya que solo aplica 2 direcciones de covarience cuando es muy redonda y luego least square, que también podría ser covarience con solo una dirección, en caso contrario, es decir, cuando hay direccionamiento del espectro de fourier)


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate
from scipy.ndimage import binary_opening

# Filtrar puntos dispersos en la máscara
def clean_mask(mask):
    cleaned_mask = binary_opening(mask, structure=np.ones((3, 3)))  # Filtro morfológico
    return cleaned_mask

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Filtro por porcentaje de energía
def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)  # Energía total de la imagen en la frecuencia
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]  # Ordenamos de mayor a menor
    cumulative_energy = np.cumsum(sorted_magnitude)  # Suma acumulativa de energía
    
    # Encontramos el umbral donde alcanzamos el porcentaje de energía deseado
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]

    # Creamos la máscara conservando solo los valores con suficiente energía
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask  # Aplicamos el filtro
    
    print(f"Umbral de energía aplicado: {threshold_value:.4f} (Conserva {energy_percent}% de la energía)")
    return filtered_transform, mask

# Calcular la dirección principal del espectro filtrado
def calculate_main_directions(mask, magnitude=None, num_directions=3):
    """
    Calcula las direcciones principales del espectro filtrado.
    
    Parámetros:
        mask (ndarray): Máscara binaria del espectro.
        magnitude (ndarray, opcional): Magnitud del espectro para ponderar los puntos.
        num_directions (int): Número de direcciones principales a calcular.
    
    Retorna:
        list: Lista de direcciones principales en grados.
    """
    h, w = mask.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Filtrar puntos dispersos
    mask = clean_mask(mask)

    # Coordenadas de los puntos activados en la máscara
    y, x = np.nonzero(mask)
    if len(x) == 0 or len(y) == 0:
        print("La máscara no tiene puntos activados.")
        return [0]  # O algún valor predeterminado

    y = y - cy
    x = x - cx

    # Calcular la relación de aspecto de los puntos activados
    range_x = np.max(x) - np.min(x)
    range_y = np.max(y) - np.min(y)
    aspect_ratio = max(range_x, 1) / max(range_y, 1)  # Evitar división por cero

    # Depuración: Imprimir los valores
    print(f"range_x: {range_x}, range_y: {range_y}, aspect_ratio: {aspect_ratio}")


    # Decidir el método basado en la relación de aspecto
    if aspect_ratio > 1.7 or aspect_ratio < 0.6: # Una sola línea preferente
        print("Usando método: least_squares")
        # Ajustar una línea usando mínimos cuadrados
        A = np.vstack([x, np.ones(len(x))]).T
        m, _ = np.linalg.lstsq(A, y, rcond=None)[0]
        main_direction = np.arctan(m)  # Ángulo en radianes
        return [np.degrees(main_direction)]  # Convertir a grados y devolver en lista
    else:
        print("Usando método: covariance")
        # Calcular la matriz de covarianza
        if magnitude is not None:
            magnitude = np.abs(magnitude)  # Asegurarse de que no haya valores negativos
            weights = magnitude[mask]
            weights /= np.sum(weights)  # Normalizar los pesos
            cov_matrix = np.cov(x, y, aweights=weights)
        else:
            cov_matrix = np.cov(x, y)

        # Calcular valores y vectores propios
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        # Ordenar los valores propios y vectores propios en orden descendente
        sorted_indices = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, sorted_indices]

        # Calcular las direcciones principales
        directions = []
        for i in range(min(num_directions, len(eigenvalues))):
            angle = np.arctan2(eigenvectors[1, i], eigenvectors[0, i])  # Ángulo en radianes
            directions.append(np.degrees(angle))  # Convertir a grados

        # Si no hay suficientes direcciones, rellena con direcciones uniformemente distribuidas
        while len(directions) < num_directions:
            directions.append((360 / num_directions) * len(directions))

        return directions

# Filtro de dos líneas delgadas rotadas
def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Crear una máscara con dos líneas delgadas
    mask = np.zeros((h, w), dtype=np.uint8)

    # Línea horizontal
    mask[cy - 1:cy + 2, :] = 1  # Grosor de 3 píxeles

    # Línea vertical
    mask[:, cx - 1:cx + 2] = 1  # Grosor de 3 píxeles

    # Rotar la máscara
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

# Función principal para procesar todas las imágenes en una carpeta
def process_images_with_rotated_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Transformada de Fourier 2D
        
        # Aplicar el filtro por energía al 20%
        filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, energy_percent=20)

        # Calcular las direcciones principales del espectro filtrado
        main_directions = calculate_main_directions(mask_energy, magnitude=np.abs(f_transform_shifted), num_directions=3)
        print(f"Direcciones principales calculadas: {main_directions}")

        # Aplicar el filtro de dos líneas rotadas para cada dirección
        combined_filtered_transform = np.zeros_like(f_transform_shifted)
        combined_rotated_mask = np.zeros_like(mask_energy, dtype=np.uint8)  # Asegurar que sea uint8
        for direction in main_directions:
            filtered_transform_lines, rotated_mask = filter_two_lines_rotated(f_transform_shifted, direction)
            combined_filtered_transform += filtered_transform_lines
            combined_rotated_mask += rotated_mask

        # Espectros filtrados para visualizar
        filtered_magnitude_spectrum_energy = np.log1p(np.abs(filtered_transform_energy))
        filtered_magnitude_spectrum_lines = np.log1p(np.abs(combined_filtered_transform))
        
        # Reconstrucción de la imagen desde la transformada filtrada
        reconstructed_img_energy = np.abs(ifftn(filtered_transform_energy))
        reconstructed_img_lines = np.abs(ifftn(combined_filtered_transform))

        # Visualización de resultados
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(2, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='turbo')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(2, 4, 3)
        plt.imshow(mask_energy, cmap='gray')
        plt.title('Máscara del Filtro (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 4)
        plt.imshow(filtered_magnitude_spectrum_energy, cmap='turbo')
        plt.title('Espectro Filtrado (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 5)
        plt.imshow(combined_rotated_mask, cmap='gray')
        plt.title(f'Máscara Rotada ({len(main_directions)} direcciones)')
        plt.axis('off')
        
        plt.subplot(2, 4, 6)
        plt.imshow(filtered_magnitude_spectrum_lines, cmap='turbo')
        plt.title('Espectro Filtrado (Líneas Rotadas)')
        plt.axis('off')
        
        plt.subplot(2, 4, 7)
        plt.imshow(reconstructed_img_energy, cmap='gray')
        plt.title('Imagen Reconstruida (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 8)
        plt.imshow(reconstructed_img_lines, cmap='gray')
        plt.title('Imagen Reconstruida (Líneas Rotadas)')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
# Ejecutar el procesamiento con filtro rotado
process_images_with_rotated_filter("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")

Ahora viene otra parte importante y diferente a la anterior. Consiste en utilizar la transformada de Hough para ver las líneas que componen la imagen reconstruida y así poder ver los puntos de intersección. Con ello luego podremos determinar las ventanas de píxeles para lo del analisis de orientation.py
